In [1]:
from IPython.display import display, Markdown, Latex

In [2]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from satellite import *

import matplotlib.pyplot as plt; import numpy as np
import geopandas as gpd; from shapely.ops import cascaded_union

import pandas as pd
pd.options.display.max_rows = 160

In [3]:
## Algunos resultados de mezclar Censo con EPH y  circuitos electorales

In [4]:
# Cargamos poligonos de mapas
gdf2 = gpd.read_file('./../EleccionesARG/data/circuitos/custom/BA_custom_areas.shp')
gdf2 = gdf2.drop(gdf2.loc[gdf2.area_0 == 'san fernando'].geometry.area.sort_values().tail(3).index)
# # numero de electores
# electoresxlugar = pd.read_csv('./data/n_electores.csv', index_col=[0, 1])

In [5]:
# electores_area0 = electoresxlugar.groupby(level = 0).sum()

In [6]:
# cat_ocups = pd.read_csv('./data/cat_ocups.csv', index_col=[0], skiprows=3)
# cat_ocups.columns = ['x', 'Desocupado', 'y','Jubilado', 'Estudiante', 'Ama de casa', 'Otros', 'Rentista', 'Cta Propia', 'Obrero', 'z', 'Patron']
# cat_ocups = cat_ocups.reindex(cat_ocups.sum().sort_values(ascending = False).index[:6], axis = 1).fillna(1)
# cat_ocups = cat_ocups
# cat_ocups['otros/err'] = 100-cat_ocups.sum(1)

# cat_ocups_ = pd.concat([cat_ocups, electores_area0], 1)

# show = cat_ocups_.sample(20).sort_values(by = 'Obrero').round(1)
# show
# import seaborn as sns
# cm = sns.light_palette("green", as_cmap=True)
# s = show.style.background_gradient(cmap=cm, subset = ['Obrero', 'Jubilado'])
# s


# # En el interior hay una piramide un poquito mas envejecida

In [7]:
votos = pd.read_csv('./data/votos_area_pct.csv', index_col=[0], header = [0])

votos.columns = ['ELECTORES', 'ELECTORES', 'ELECTORES', 'VOTANTES', 'VOTANTES',
       'VOTANTES', 'CAMBIEMOS', 'CAMBIEMOS', 'CAMBIEMOS', 'FPV / UC', 'FPV / UC', 'FPV / UC', 'Massa', 'Massa', 'Massa', 'BLANCOS', 'BLANCOS', 'BLANCOS',
       'FJ (Randazzo)', 'FJ (Randazzo)', 'FIT', 'FIT', 'FIT']
cols = list(zip(votos.columns, votos.iloc[0,:]))
c1 = pd.MultiIndex.from_tuples(cols, names=['Fuerza', 'Eleccion'])
votos = pd.DataFrame(data=votos[2:].values, columns=c1, index=votos.index[2:])
votos.index.names = ['area_0']
votos = votos.astype(float).round(1)#.astype(int)

# votos.sample(3).sample(3, axis = 1)


In [8]:
### Dataset votos

In [9]:

# votos = pd.read_csv('./../EleccionesARG/data/tablas_votos/VOTOS_2015_17_circ_cnt.csv', index_col=[0, 1, 2], header = [0])
votos = pd.read_csv('./data/votos_comb.csv', index_col=[0, 1], header = [0])
# Toda esta vuelta es para ponerle nombres familiares a las listas. Pandas no deja setear nombres repetidos en nivel de multiindex
votos.columns = ['ELECTORES', 'ELECTORES', 'ELECTORES', 'VOTANTES', 'VOTANTES',
       'VOTANTES', 'CAMBIEMOS', 'CAMBIEMOS', 'CAMBIEMOS', 'FPV / UC', 'FPV / UC', 'FPV / UC', 'Massa', 'Massa', 'Massa', 'BLANCOS', 'BLANCOS', 'BLANCOS',
       'FJ (Randazzo)', 'FJ (Randazzo)', 'FIT', 'FIT', 'FIT']
cols = list(zip(votos.columns, votos.iloc[0,:]))
c1 = pd.MultiIndex.from_tuples(cols, names=['Fuerza', 'Eleccion'])
votos = pd.DataFrame(data=votos[2:].values, columns=c1, index=votos.index[2:])
votos.index.names = ['area_0','area_1']
votos = votos.astype(float).astype(int)
# votos.head()

In [10]:
## Ej. ingresos por circuito

In [11]:
# Ingresos por partido y circuito. Se hace proyectando resultados EPH (Indec 2016-18) con la base individual del Censo 2010.
# Se cruzan los poligonos de radios censales a distritos electorales.
# Pesos deflactados en base a cavallo/pricestats. Serian pesos corrientes a nivel de Sep. 18.
# Para 'proyectar' la encuenta permanente de hogares con el censo se usa un modelo de random forests (machine learning). 
# En base a las caracteristicas de la persona y la vivienda segun censo se predice ingreso con info de EPH

In [12]:
result_comb = pd.read_csv('./data/result_comb_circ.csv', header = [0, 1], index_col = [0, 1])
show = result_comb.sort_values(by = ('Ingresos (basado en EPH)','electores')).tail(30).sort_index().sort_values(by = ('Ingresos (basado en EPH)','mediana'))
# show
# s = show.style.background_gradient(cmap=cm)
# s

# # # Ingresos medios por distrito. Esto esta graficado vs votos en los scatterplots

In [13]:
# %%latex
# \newpage

# display(Markdown('Meter markdown en el medio del codigo. Ejemplo plot en un loop y en el medio le ponemos un comentario'))


In [14]:
display(Markdown('# Atlas Distritos'))
display(Markdown('## Consulta de distritos electorales'))


# Atlas Distritos

## Consulta de distritos electorales

In [15]:
%%latex
\newpage

<IPython.core.display.Latex object>

In [16]:
display(Markdown('## Indice.'))
display(Markdown('Ordenados por cantidad de electores. Aqui porc. y acumulado de electores por partido.'))
display(Markdown('***Si no se encuentra, buscar con Ctrl+F***'))
electores_por_area = votos.groupby(axis = 0, level = 0).sum()[[('ELECTORES', 'DN_2017')]]
sorted_ = electores_por_area.sort_values(by = [('ELECTORES', 'DN_2017')], ascending = False)
orden = (100 * sorted_/sorted_.sum()); orden.columns = ['porcentaje (%)']
orden['acumulado (%)'] = orden['porcentaje (%)'].cumsum()
orden.round(1)#.loc[pd.Index([i for i in orden.index if 'Secci' not in i])]

## Indice.

Ordenados por cantidad de electores. Aqui porc. y acumulado de electores por partido.

***Si no se encuentra, buscar con Ctrl+F***

,porcentaje (%),acumulado (%)
area_0,,
la matanza,7.0,7.0
la plata,3.7,10.7
general pueyrredon,3.7,14.4
lomas de zamora,3.4,17.8
quilmes,3.1,20.9
almirante brown,2.9,23.8
merlo,2.7,26.5
lanus,2.6,29.1
Resto 5a Seccion,2.5,31.6


In [17]:
%%latex
\newpage

<IPython.core.display.Latex object>

In [19]:

for area in orden.index.values:
# for area in ['la plata', 'general pueyrredon', 'bahia blanca']:

    display(Markdown('## Lugar: '))
    print(area)
    #Mostrar ingresos por circuito
    display(Markdown('#### Percentil ingresos por circuito: '))
    display(Markdown('Ingresos en pesos corrientes de Sep-2018. Quantiles 25%, 75% y mediana.'))

    display(result_comb.loc[area].sort_values(by = ('Ingresos (basado en EPH)','mediana')))
    
    area_df = gdf2.loc[gdf2.area_0 == area]
    
    geom = area_df['geometry']
    
    if area[9:] == u'Secci':
        zoom = 10
    else:
        zf = 13 - 1.5*np.log(gpd.GeoSeries(cascaded_union(area_df['geometry'])).length)
        zoom = int(round(zf))
#     print(zoom)

#     if (area == 'la plata'): zoom = 13
#     if (area == 'general pueyrredon'): zoom = 13
        
    bounds = gpd.GeoSeries(cascaded_union(area_df['geometry'])).bounds.loc[0]
        
    final = get_maps_image(bounds, zoom = zoom)
    # Poly
    scale_x = final.width/(bounds[2] - bounds[0])
    scale_y = final.height/(bounds[3] - bounds[1])

    scaled = area_df.scale(xfact=scale_x, yfact=-scale_y, origin=(0, 0))
    # Translate (we need to know the center)
    scaled_area = gpd.GeoSeries(cascaded_union(scaled))
    centerx, centery = (scaled_area.bounds['maxx'] + scaled_area.bounds['minx'])/2., (scaled_area.bounds['maxy'] + scaled_area.bounds['miny'])/2.
    trans = scaled.translate(final.width/2 - centerx,final.height/2 - centery)

    display(Markdown('#### Mapa: '))

    fig, ax = plt.subplots(1, figsize = (15, 16))
    ax.imshow(final)
    ax.axis('off')
 
    area_df['coords'] = trans.apply(lambda x: x.representative_point().coords[:])
    area_df['coords'] = [coords[0] for coords in area_df['coords']]

    minx, maxx = 0, final.width
    miny, maxy = final.height, 0
    
#     print(final.width)
#     print(final.height)
    offset = -30
    
    if (area == 'la plata'):
        q = .15 # Hackeo para que se vean bien 
        minx, maxx = 720, 1200
        miny, maxy = 2821, 2400
        offset = -60
        
    if (area == 'general pueyrredon'):
        q = .15 # Hackeo para que se vean bien 
        minx, maxx = 1000, 1600
        miny, maxy = 1600, 700
        offset = -5
        
    if (area == 'bahia blanca'):
        q = .15 # Hackeo para que se vean bien 
        minx, maxx = 700, 1300
        miny, maxy = 1690, 1000
        
    if (area == 'comuna 04, tigre, Resto 6a Seccion'): offset = -50
    if (area in 'avellaneda, berazategui, san miguel, comuna 13, comuna 14, comuna 01, comuna 02, campana, Resto 5a Seccion, zarate'): offset = 0
        
    trans.translate(0, offset).plot(ax = ax, alpha = 1, facecolor="none", edgecolor='r', linewidth = 1.5) #color = '#ff7f0e',
    for idx, row in area_df.iterrows():
        x, y = row['coords']; y += offset
        if (minx <= x <= maxx) and (maxy <= y <= miny):
            ax.annotate(s = row['area_1'], xy = (x, y), horizontalalignment='center', fontsize = 15, color = 'k', fontweight='bold', backgroundcolor = '.9', alpha = .7)
    
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy) 
    plt.show()
    
    display(Markdown('### Numeros de votos. 2015-17: '))

    ### Tabla votos:
    votos_area = votos.loc[area].astype(int)#.stack()
    max_votos = votos_area.groupby(axis = 1, level = 0).max()
    orden = max_votos.mean().sort_values(ascending = False).head().index
    votos_area = votos_area.stack()[orden]
    votos_area['Resto'] = votos_area.VOTANTES - votos_area.iloc[:, 2:].sum(1)

    ix = votos_area['ELECTORES'].sort_values().tail(21).index.get_level_values(0).drop_duplicates()
    votos_area_show = votos_area.loc[ix]
    votos_area_show = votos_area_show.reindex(pd.MultiIndex.from_product([list(ix), ['DN_2017', 'paso_2017', 'DN_2015']]))
    votos_area_show
    display(Markdown('votos por circuito. (puede omitir pequenos)'))

    display(votos_area_show.fillna(''))


*Matias Iglesias*